## 1. Importi

In [ ]:
from collections import defaultdict
import json
import os.path
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplcyberpunk
plt.style.use("cyberpunk")


## 2. Branje podatkov

In [ ]:
with open('../data/kd2018.csv', 'rt', encoding="cp1250") as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    head0 = next(reader)
    none_handler = lambda i : i or None
    main_data18 = np.array([none_handler(i) for i in [row for row in reader]])

In [ ]:
with open('../data/kd2019.csv', 'rt', encoding="cp1250") as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    head1 = next(reader)
    none_handler = lambda i : i or None
    main_data19 = np.array([none_handler(i) for i in [row for row in reader]])

In [ ]:
with open('../data/kd2020.csv', 'rt', encoding="cp1250") as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    head2 = next(reader)
    none_handler = lambda i : i or None
    main_data20 = np.array([none_handler(i) for i in [row for row in reader]])

In [ ]:
with open('../data/kd2021.csv', 'rt', encoding="cp1250") as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    head3 = next(reader)
    none_handler = lambda i : i or None
    main_data21 = np.array([none_handler(i) for i in [row for row in reader]])

In [ ]:
with open('../data/obcine_slovenije.csv', 'rt', encoding='utf8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    head_obcine = next(reader)
    none_handler = lambda i : i or None
    obcine = np.array([none_handler(i) for i in [row for row in reader]])

## 3. Priprava podatkov

In [ ]:
dict_obcina_regija = dict((str.lower(x[0]), str.lower(x[7])) for x in obcine)

Odstranjevanje duplikatov:

In [ ]:
found_ids = set()
rows_to_keep = np.array([])
for i in range(len(main_data18)):
    if (main_data18[i][0] not in found_ids):
        rows_to_keep = np.append(rows_to_keep, True)
        found_ids.add(main_data18[i][0])
    else:
        rows_to_keep = np.append(rows_to_keep, False)

main_data18_filtered = main_data18[np.ma.make_mask(rows_to_keep)]

found_ids = set()
rows_to_keep = np.array([])
for i in range(len(main_data19)):
    if (main_data19[i][0] not in found_ids):
        rows_to_keep = np.append(rows_to_keep, True)
        found_ids.add(main_data19[i][0])
    else:
        rows_to_keep = np.append(rows_to_keep, False)

main_data19_filtered = main_data19[np.ma.make_mask(rows_to_keep)]

found_ids = set()
rows_to_keep = np.array([])
for i in range(len(main_data20)):
    if (main_data20[i][0] not in found_ids):
        rows_to_keep = np.append(rows_to_keep, True)
        found_ids.add(main_data20[i][0])
    else:
        rows_to_keep = np.append(rows_to_keep, False)

main_data20_filtered = main_data20[np.ma.make_mask(rows_to_keep)]

found_ids = set()
rows_to_keep = np.array([])
for i in range(len(main_data21)):
    if (main_data21[i][0] not in found_ids):
        rows_to_keep = np.append(rows_to_keep, True)
        found_ids.add(main_data21[i][0])
    else:
        rows_to_keep = np.append(rows_to_keep, False)

main_data21_filtered = main_data21[np.ma.make_mask(rows_to_keep)]

main_data = np.concatenate((main_data18_filtered, main_data19_filtered, main_data20_filtered, main_data21_filtered))

Izračunana vrednost "nevarnosti" za vse zločine s formulo: 
$$ x = ((max\_jailtime + avg\_actual\_jailtime) * dangerousness) $$
V primeru, da je avg_actual_jailtime = NaN, namesto tega dodam max_jailtime, saj to pomeni, da se v naših podatkih zločinov ta zločin ni nikoli zgodil in predpostavimo, da je to zelo hud zločin. Primer takega zločina je "genocid".

In [ ]:
# read json data from kazni_dict.json
with open('../data/kazni_dict.json', 'r', encoding='utf8') as f:
    kazni_dict = json.load(f)

In [ ]:
kazni_utezi = dict()
for kd, values in kazni_dict.items():
        if (pd.isna(values['average_actual_jailtime'])):
            x = values['max_jailtime'] * 2 * values['dangerousness']
        else:
            x = (values['max_jailtime'] + values['average_actual_jailtime']) * values['dangerousness']
        kazni_utezi[kd] = x

In [ ]:
regije = dict.fromkeys([str.lower(x) for x in np.unique(obcine[:, 7])], 0)
city_errors = set()
kd_errors_dict = defaultdict(int)


for kd in main_data:
    city = str.lower(kd[19])
    kd = str.lower(str.strip(str.split(kd[6], '-')[-1]))
    if (city == 'neznana ue' or city == 'neznana obč' or city == 'ni podatka' or kd == 'poneverba in neupravičena uporaba tujega premoženja' or kd == ''):
        continue
    if (city == 'šentjur pri celju'):
        city = 'šentjur'
    try:
        regije[dict_obcina_regija[city]] += kazni_utezi[kd]
    except KeyError:
        if (city not in dict_obcina_regija):
            city_errors.add(city)
            print(f"NAPAKA PRI \"{city}\"")
        else:
            kd_errors_dict[kd] += 1
            print(f"NAPAKA PRI \"{kd}\"")

# kd_errors = np.array(list(kd_errors_dict.items()), dtype=object)
# kd_errors = kd_errors[np.argsort(kd_errors[:, 1])[::-1]]
# kd_errors

In [ ]:
np_regije_danger = np.array(list(regije.items()), dtype=object)
np_regije_danger = np_regije_danger[np.argsort(np_regije_danger[:, 1])[::-1]]

# create graph of dangerousness of regions
fig, ax = plt.subplots()
ax.barh(np_regije_danger[:, 0], np_regije_danger[:, 1])
ax.set_xlabel('Dangerousness')
ax.set_ylabel('Regions')
ax.set_title('Dangerousness of regions')
plt.show()


Sedaj delim rezultate z številom prebivalcev. Število prebivalcev datuma 1. julij (2020).

In [ ]:
regije_prebivalci = dict()
with open('../data/regije_prebivalci.csv', 'rt', encoding='utf8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    head_regije_prebivalci = next(reader)
    none_handler = lambda i : i or None
    for x in reader:
        x = x[0].split(';')
        regije_prebivalci[x[0]] = int(x[1])

In [ ]:
np_regije_danger_1 = np.array([[x[0], round(x[1] / regije_prebivalci[x[0]], 5)] for x in np_regije_danger], dtype=object)

In [ ]:
np_regije_danger_1 = np_regije_danger_1[np.argsort(np_regije_danger_1[:, 1])[::-1]]

# create graph of dangerousness of regions
fig, ax = plt.subplots()
ax.barh(np_regije_danger_1[:, 0], np_regije_danger_1[:, 1])
ax.set_xlabel('Dangerousness')
ax.set_ylabel('Regions')
ax.set_title('Dangerousness of regions')
plt.show()

## 4. Razložitev rezultatov

Vrednost predstavlja, kako nevarni

## 5. Exporting stuff

In [ ]:
export_dict: dict = {"regions": dict.fromkeys(np_regije_danger[:,0], {})}

In [ ]:
export_dict["regions"] = {k: {"data_danger_score": v*100,
                              "data_names": [
                                  "Najpogostejši kriminal",
                                  "Najpogostejši kriminal"
                              ],
                              "data_values": [
                                  "Tatvina",
                                  "Tatvina"
                              ], } for k, v in np_regije_danger_1}


### Fix regions that don't match the Nominatim API

In [ ]:
export_dict["regions"]["obalno - kraška"] = export_dict["regions"]["obalno-kraška"]
del export_dict["regions"]["obalno-kraška"]
export_dict["regions"]["jugovzhodna slovenija"] = export_dict["regions"]["jugovzhodna"]
del export_dict["regions"]["jugovzhodna"]
export_dict["regions"]["primorsko - notranjska"] = export_dict["regions"]["primorsko-notranjska"]
del export_dict["regions"]["primorsko-notranjska"]

In [ ]:
json.dump(export_dict, open('../data/regije_app_data.json', 'w'), ensure_ascii=False, indent=4)